# UN Sustainable Development Goals

This notebook implements the pre-processing needed for importing the UN SDG dataset into OWID's grapher database.
A rough outline of the process:

  1. Read the dataset exported from the UN SDG Indicators database website [[1]](#Data-loading-and-preprocessing)
  2. Export the referenced _entitites_ (geographic areas) [[2]](#Export-entities-(dimension-members))
  3. _Reconcile_ those entities with OpenRefine and [OWID's geographic entities reconciliation service](https://github.com/owid/lc-reconcile/)
  4. Generate a separate table for every combination distinct values of geographic entities and other nominal variables ([3](#Export-datasets-and-variables))
  5. Export a `variables.csv` file, and a set of `dataset_*.csv` files that contains each generated table. [[4]](#Export-data)

In [119]:
import pandas as pd
import numpy as np
import collections
import itertools
import functools
import requests
from pandas.io.json import json_normalize


pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', -1)

In [102]:
definitions = pd.read_excel('data/SDG-DSD-Matrix.xlsx', None)

In [103]:
definitions.keys()

odict_keys(['Data Model', 'CL_ACTIVITY', 'CL_AGE', 'CL_AREA', 'CL_COMP_BREAKDOWN', 'CL_CUST_BREAKDOWN', 'CL_DISABILITY', 'CL_EDU_LEVEL', 'CL_FREQ', 'CL_GEO_INFO_TYPE', 'CL_NATURE', 'CL_OBS_STATUS', 'CL_OCCUPATION', 'CL_PRODUCT', 'CL_QUANTILE', 'CL_REPORTING_TYPE', 'CL_SERIES', 'CL_SEX', 'CL_UNIT_MEASURE', 'CL_UNIT_MULT', 'CL_URBANISATION'])

In [143]:
codelists = requests.get('https://unstats.un.org/SDGAPI/v1/sdg/Goal/Data').json()['dimensions']

In [151]:
[c['id'] for c in codelists]

['Age',
 'Cities',
 'Disability status',
 'Education level',
 'Freq',
 'Hazard type',
 'IHR Capacity',
 'Level/Status',
 'Location',
 'Migratory status',
 'Mode of transportation',
 'Name of international agreement',
 'Name of international institution',
 'Name of non-communicable disease',
 'Policy Domains',
 'Quantile',
 'Reporting Type',
 'Sex',
 'Tariff regime (status)',
 'Type of facilities',
 'Type of mobile technology',
 'Type of occupation',
 'Type of product',
 'Type of skill',
 'Type of speed']

In [146]:
codelist_dict = ({ codelist['id']: pd.DataFrame(codelist['codes']) for codelist in codelists })

In [147]:
codelist_dict['Hazard type']

,code,description,sdmx
0,ACIDT,Accident,HZT_ACIDT
1,ACIDR,Acid rain,HZT_ACIDR
2,ALLUV,Alluvion,HZT_ALLUV
3,ANIAK,Animal Attack,HZT_ANIAK
4,AVALE,Avalanche,HZT_AVALE
5,BIOGL,Biological,HZT_BIOGL
6,CHESP,Chemical Spill,HZT_CHESP
7,COSER,Coastal Erosion,HZT_COSER
8,COSFL,Coastal Flood,HZT_COSFL
9,COLDW,Cold Wave,HZT_COLDW


## Data loading and preprocessing

The data was obtained from the [UN SDG Indicators database](https://unstats.un.org/sdgs/indicators/database). We selected all _Goals_ (topmost category in the classification of indicators) and requested the entire dataset. 

In [2]:
data = pd.read_csv(
    "data/20190903150325064_drifter4e@gmail.com_data.csv", low_memory=False
)

In [148]:
DIMENSIONS = [c[1:-1] for c in data.columns if c[0] == '[' and c[-1] == ']']

In [156]:
pd.merge(pd.DataFrame({ 'dimensions': DIMENSIONS }), pd.DataFrame({ 'codelists': list(codelist_dict.keys()) }), left_on='dimensions', right_on='codelists')

,dimensions,codelists
0,Age,Age
1,Cities,Cities
2,Disability status,Disability status
3,Education level,Education level
4,Hazard type,Hazard type
5,IHR Capacity,IHR Capacity
6,Level/Status,Level/Status
7,Location,Location
8,Migratory status,Migratory status
9,Mode of transportation,Mode of transportation


In [150]:
[ codelist_dict[dim] for dim in DIMENSIONS]

KeyError: 'Bounds'

In [68]:
dim = '[Hazard type]'

In [83]:
data[dim].dropna().drop_duplicates().sort_values().reset_index(drop=True)

0     ACIDR
1     ACIDT
2     ALLUV
3     ANIAK
4     AVALE
5     BIOGL
6     CHESP
7     COLDW
8     CONTM
9     COSER
10    COSFL
11    CSOLD
12    CYCLN
13    DFRST
14    DROUG
15     DZUD
16    ELEST
17    EPIDM
18    EPIZT
19    EROSN
20    ERQAK
21    EXPLN
22    EXTEM
23    FIREX
24    FLOOD
25    FLSFL
26    FOGXX
27    FROST
28    FRZRN
29    GLCFL
30    HAILS
31    HETWA
32    HEVRN
33    INFET
34    INTOX
35    LAHAR
36    LIQFN
37    LNDRG
38    LNDSL
39    NUCIN
40    OTHER
41    PANIC
42    PEATX
43    PESTX
44    PLGUE
45    POLUT
46    SANDS
47    SEDMN
48    SNSTM
49    STCOL
50    STORM
51    STUCK
52    SUBSD
53    TECHH
54    TORND
55    TSUNM
56    VOLER
57    WLDFR
58    WNDST
Name: [Hazard type], dtype: object

In [90]:
data[data['[Migratory status]'] == '_T']

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,Source,FootNote,Nature,Units,[Age],[Bounds],[Cities],[Disability status],[Education level],[Hazard type],[IHR Capacity],[Level/Status],[Location],[Migratory status],[Mode of transportation],[Name of international institution],[Name of non-communicable disease],[Quantile],[Reporting Type],[Sex],[Tariff regime (status)],[Type of mobile technology],[Type of occupation],[Type of product],[Type of skill],[Type of speed]
701777,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",12,Algeria,2000,27.28,2000,ILOSTAT - ADM-IR - Insurance records,Coverage of occupational injuries: Compensated...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
701778,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",12,Algeria,2001,23.24,2001,ILOSTAT - ADM-IR - Insurance records,Coverage of occupational injuries: Compensated...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
701779,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",12,Algeria,2002,20.88,2002,ILOSTAT - ADM-IR - Insurance records,Coverage of occupational injuries: Compensated...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
701780,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",12,Algeria,2003,19.03,2003,ILOSTAT - ADM-IR - Insurance records,Coverage of occupational injuries: Compensated...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
701781,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",12,Algeria,2004,17.59,2004,ILOSTAT - ADM-IR - Insurance records,Coverage of occupational injuries: Compensated...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
701782,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",31,Azerbaijan,2000,4,2000,ILOSTAT - ADM-LIR - Labour inspectorate records,Coverage of occupational injuries: Reported in...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
701783,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",31,Azerbaijan,2001,3,2001,ILOSTAT - ADM-LIR - Labour inspectorate records,Coverage of occupational injuries: Reported in...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
701784,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",31,Azerbaijan,2001,5,2001,ILOSTAT - ADM-LIR - Labour inspectorate records,Coverage of occupational injuries: Reported in...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,MALE,NaN,NaN,NaN,NaN,NaN,NaN
701785,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",31,Azerbaijan,2001,0,2001,ILOSTAT - ADM-LIR - Labour inspectorate records,Coverage of occupational injuries: Reported in...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,FEMALE,NaN,NaN,NaN,NaN,NaN,NaN
701786,8,8.8,8.8.1,SL_EMP_FTLINJUR,"Fatal occupational injuries among employees, b...",31,Azerbaijan,2002,0,2002,ILOSTAT - ADM-LIR - Labour inspectorate records,Coverage of occupational injuries: Reported in...,C,PER_100000_EMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,_T,NaN,NaN,NaN,NaN,G,FEMALE,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
data[data['[Bounds]'] == 'MP']

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,Source,FootNote,Nature,Units,[Age],[Bounds],[Cities],[Disability status],[Education level],[Hazard type],[IHR Capacity],[Level/Status],[Location],[Migratory status],[Mode of transportation],[Name of international institution],[Name of non-communicable disease],[Quantile],[Reporting Type],[Sex],[Tariff regime (status)],[Type of mobile technology],[Type of occupation],[Type of product],[Type of skill],[Type of speed]
88577,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),1,World,2015,8.7,2015,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The value(s) for one or more years may have been imputed. The observation value represents the 3-year average of the period 2014-2016.,E,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88579,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),1,World,2016,9.2,2016,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The value(s) for one or more years may have been imputed. The observation value represents the 3-year average of the period 2015-2017.,E,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88582,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),2,Africa,2015,23.4,2015,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The observation value represents the 3-year average of the period 2014-2016.,E,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88585,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),2,Africa,2016,25.9,2016,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The observation value represents the 3-year average of the period 2015-2017.,E,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88589,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),4,Afghanistan,2015,15.4,2015,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The observation value represents the 3-year average of the period 2014-2016.,G,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88593,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),4,Afghanistan,2016,16,2016,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The observation value represents the 3-year average of the period 2015-2017.,G,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88596,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),5,South America,2015,5.8,2015,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The observation value represents the 3-year average of the period 2014-2016.,E,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88599,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of severe food insecurity in the adult population (%),5,South America,2016,6.9,2016,"Food and Agriculture Organisation of the United Nations (FAO), with data collected through the Gallup World Poll.",The observation value represents the 3-year average of the period 2015-2017.,E,PERCENT,ALLAGE,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88602,2,2.1,2.1.2,AG_PRD_FIESSI,Prevalence of se

In [91]:
all_dimensions = pd.DataFrame([data[dim].dropna().drop_duplicates().sort_values().reset_index(drop=True) for dim in DIMENSIONS]).T

In [110]:
all_dimensions

,[Age],[Bounds],[Cities],[Disability status],[Education level],[Hazard type],[IHR Capacity],[Level/Status],[Location],[Migratory status],[Mode of transportation],[Name of international institution],[Name of non-communicable disease],[Quantile],[Reporting Type],[Sex],[Tariff regime (status)],[Type of mobile technology],[Type of occupation],[Type of product],[Type of skill],[Type of speed]
0,1-14,LB,ABIDJAN,PD,GRAD23,ACIDR,IHR01,HIGIMP,ALLAREA,EUMIG,AIR,ADB,CAN,NaN,G,BOTHSEX,MFN,AL2G,DENT,AGR,ARSP,10MBPS
1,1-4,MP,ACAPULCO_GUERRERO,PWD,LOWSEC,ACIDT,IHR02,LOWIMP,RURAL,MIGPER,RAI,AFDB,CAR,NaN,NaN,FEMALE,PRF,AL3G,NURS,ALP,CMFL,256KT2MBPS
2,10+,UB,ACCRA,_T,PREPRI,ALLUV,IHR03,MHIGIMP,URBAN,NONEUMIG,ROA,ECOSOC,DIA,NaN,NaN,MALE,NaN,AL4G,PHAR,ARM,COPA,2MT10MBPS
3,10-14,NaN,ACUNACOAHUILA,NaN,PRIMAR,ANIAK,IHR04,MLOWIMP,NaN,NONMIG,NaN,FSB,RES,NaN,NaN,NaN,NaN,NaN,PHYS,BIM,EMAIL,ANYS
4,10-17,NaN,ADDIS,NaN,SECOND,AVALE,IHR05,VHIGIMP,NaN,_T,NaN,IABD,NaN,NaN,NaN,NaN,NaN,NaN,isco08,CLO,EPRS,NaN
5,12+,NaN,ADELAIDE,NaN,UPPSEC,BIOGL,IHR06,VLOWIMP,NaN,NaN,NaN,IBRD,NaN,NaN,NaN,NaN,NaN,NaN,isco08-0,COL,INST,NaN
6,12-14,NaN,AGUASCALIENTES,NaN,NaN,CHESP,IHR07,NaN,NaN,NaN,NaN,IFC,NaN,NaN,NaN,NaN,NaN,NaN,isco08-1,CPR,LITE,NaN
7,12-24,NaN,ALGIERS,NaN,NaN,COLDW,IHR08,NaN,NaN,NaN,NaN,IMF,NaN,NaN,NaN,NaN,NaN,NaN,isco08-2,CRO,NUME,NaN
8,14+,NaN,ALTAMIRA_TAMAULIPAS,NaN,NaN,CONTM,IHR09,NaN,NaN,NaN,NaN,UNGA,NaN,NaN,NaN,NaN,NaN,NaN,isco08-3,FEO,PCPR,NaN
9,14-28,NaN,AMSTERDAM-UTRECHT,NaN,NaN,COSER,IHR10,NaN,NaN,NaN,NaN,UNSC,NaN,NaN,NaN,NaN,NaN,NaN,isco08-4,FOF,SOFT,NaN


In [101]:
data[data.Indicator == '8.8.1'][['SeriesDescription']]

,SeriesDescription
701777,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701778,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701779,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701780,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701781,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701782,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701783,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701784,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701785,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"
701786,"Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees)"


Keep the indicators that we care about (list taken from [the old importer](https://github.com/owid/owid-importer/blob/master/importer_django/un_sdg_importer.py)).

In [39]:
INDICATORS = [
'1.1.1','1.2.1','1.3.1','1.5.1','1.5.2','1.5.3','2.1.1','2.1.2','2.2.1','2.2.2','2.5.1','2.5.2','2.a.1','2.a.2','2.c.1','3.1.1','3.1.2','3.2.1','3.2.2','3.3.1','3.3.2','3.3.3','3.3.5','3.4.1','3.4.2','3.5.2','3.6.1','3.7.1','3.7.2','3.9.1','3.9.2','3.9.3','3.a.1','3.b.2','3.c.1','3.d.1','4.1.1','4.2.1','4.2.2','4.3.1','4.4.1','4.5.1','4.6.1','4.a.1','4.b.1','4.c.1','5.2.1','5.3.1','5.3.2','5.4.1','5.5.1','5.5.2','5.6.1','5.b.1','6.1.1','6.2.1','6.4.2','6.5.1','6.a.1','6.b.1','7.1.1','7.1.2','7.2.1','7.3.1','8.1.1','8.2.1','8.3.1','8.4.1','8.4.2','8.5.1','8.5.2','8.6.1','8.7.1','8.8.1','8.10.1','8.10.2','8.a.1','9.1.2','9.2.1','9.2.2','9.4.1','9.5.1','9.5.2','9.a.1','9.b.1','9.c.1','10.1.1','10.4.1','10.6.1','10.a.1','10.b.1','10.c.1','11.1.1','11.5.1','11.5.2','11.6.1','11.6.2','11.b.1','12.2.1','12.2.2','12.4.1','13.1.1','13.1.2','14.4.1','14.5.1','15.1.1','15.1.2','15.2.1','15.4.1','15.4.2','15.5.1','15.6.1','15.a.1','15.b.1','16.1.1','16.2.1','16.2.2','16.2.3','16.3.2','16.5.2','16.8.1','16.9.1','16.10.1','16.10.2','16.a.1','17.2.1','17.3.2','17.4.1','17.6.2','17.8.1','17.9.1','17.10.1','17.11.1','17.12.1','17.15.1','17.16.1','17.18.2','17.18.3','17.19.1','17.19.2'
]

data = data[data.Indicator.isin(INDICATORS)]


## Export datasets and variables

Algorithm outline:

  - For each `INDICATOR`:
    - Obtain dimensions (columns named `[between brackets]`) that contain non-null values
      - For each combination of unique values values in those dimensions
        - Generate a table of values.


In [40]:
DIMENSIONS = [c for c in data.columns if c[0] == '[' and c[-1] == ']']
NON_DIMENSIONS = [c for c in data.columns if c not in set(DIMENSIONS)]

@functools.lru_cache(maxsize=256)
def get_series_with_relevant_dimensions(indicator, series):
    """ For a given indicator and series, return a tuple:
    
      - data filtered to that indicator and series
      - names of relevant dimensions
      - unique values for each relevant dimension
    """
    data_filtered = data[(data.Indicator == indicator) & (data.SeriesCode == series)]
    non_null_dimensions_columns = [col for col in DIMENSIONS if data_filtered.loc[:, col].notna().any()]
    dimension_names = []
    dimension_unique_values = []
    
    for c in non_null_dimensions_columns:
        uniques = data_filtered[c].unique()
        if len(uniques) > 1:
            dimension_names.append(c)
            dimension_unique_values.append(list(uniques))

    return (data_filtered[NON_DIMENSIONS + dimension_names], dimension_names, dimension_unique_values)

Generate tables for:

  - Rows where the dimension is `None`
  - One table for each combination of unique values of relevant dimensions

In [41]:
@functools.lru_cache(maxsize=256)
def generate_tables_for_indicator_and_series(indicator, series):
    tables_by_combination = {}
    data_filtered, dimensions, dimension_values = get_series_with_relevant_dimensions(indicator, series)
    if len(dimensions) == 0:
        # no additional dimensions
        export = data_filtered
        return export
    else:
        for dimension_value_combination in itertools.product(*dimension_values):
            # build filter by reducing, start with a constant True boolean array
            filt = [True] * len(data_filtered)
            for dim_idx, dim_value in enumerate(dimension_value_combination):
                dimension_name = dimensions[dim_idx]
                value_is_nan = type(dim_value) == float and math.isnan(dim_value)
                filt = filt \
                       & (data_filtered[dimension_name].isnull() if value_is_nan else data_filtered[dimension_name] == dim_value)

            tables_by_combination[dimension_value_combination] = data_filtered[filt].drop(dimensions, axis=1)
            
        return tables_by_combination
    

In [42]:
all_series = data[['Indicator', 'SeriesCode', 'SeriesDescription', 'Units']] \
  .groupby(by=['Indicator', 'SeriesCode', 'SeriesDescription', 'Units']) \
  .count() \
  .reset_index()

### Export data

For each series and combination of additional dimensions' members, generate an entry in the `variables` table.

In [45]:
DF_COLS_VARIABLES = ['Indicator', 'SeriesCode', 'VariableDescription', 'Units', 'variable_idx']
DF_COLS_DATASETS = ['Indicator', 'SeriesCode', 'SeriesDescription']
DF_COLS_DATAPOINTS = ['Value', 'TimePeriod', 'Time_Detail', 'Source', 'FootNote', 'Nature', 'GeoAreaCode', 'GeoAreaName']
variables = pd.DataFrame(columns=DF_COLS_VARIABLES)
datasets = pd.DataFrame(columns=DF_COLS_DATASETS)

variable_idx = 0

for i, row in all_series.iterrows():
    datasets = datasets.append(
        {
            'Indicator': row['Indicator'], 
            'SeriesCode': row['SeriesCode'], 
            'SeriesDescription': row['SeriesDescription']
        }, 
        ignore_index=True)
    _, dimensions, dimension_members = get_series_with_relevant_dimensions(row['Indicator'], row['SeriesCode'])
    
    if len(dimensions) == 0:
        # no additional dimensions
        table = generate_tables_for_indicator_and_series(row['Indicator'], row['SeriesCode'])
        variable = { 
            'Indicator': row['Indicator'], 'SeriesCode': row['SeriesCode'], 
            'VariableDescription': row['SeriesDescription'], 'Units': row['Units'],
            'variable_idx': variable_idx
        }
        variables = variables.append(variable, ignore_index=True)
        table[DF_COLS_DATAPOINTS].to_csv('./exported_data/%04d_datapoints.csv' % variable_idx, index=False)
        variable_idx += 1

    else:
        # has additional dimensions
        for member_combination, table in generate_tables_for_indicator_and_series(row['Indicator'], row['SeriesCode']).items():
            variable = { 
                'Indicator': row['Indicator'], 'SeriesCode': row['SeriesCode'], 
                'Units': row['Units'],
                'VariableDescription': row['SeriesDescription'] + " %s" % ( ' - '.join(map(str, member_combination))),
                'variable_idx': variable_idx
            }
            variables = variables.append(variable, ignore_index=True)
            table[DF_COLS_DATAPOINTS].to_csv('./exported_data/%04d_datapoints.csv' % variable_idx, index=False)
            variable_idx += 1


variables.to_csv('./exported_data/variables.csv', index=False)
datasets.to_csv('./exported_data/datasets.csv', index=False)